## Developed by Jonathan Ojeda 13-12-2020 QAAFI UQ
### Function: Create, write, edit and explore NetCDF files

In [1]:
#Import required packages
import netCDF4 as nc
import numpy as np
import xarray as xr
import csv

#Extend screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Create a NetCDF Dataset
fn = 'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/data/campaign/created_campaign/test19.nc4'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

In [3]:
#Add Dimensions
lat = ds.createDimension('lat', 360)
lon = ds.createDimension('lon', 720)
scen = ds.createDimension('scen', 8)

In [4]:
#Add NetCDF Variables
lats = ds.createVariable('lat', 'f8', ('lat',))
lats.units = 'degrees_north'
lats._Storage = 'contiguous'
lons = ds.createVariable('lon', 'f8', ('lon',))
lons.units = 'degrees_east'
lons._Storage = 'contiguous'
scens = ds.createVariable('scen', 'f4', ('scen',))

automatic_irrigation = ds.createVariable('automatic_irrigation', 'i4', ('scen',))
automatic_irrigation.units = 'Mapping'
automatic_irrigation.long_name = 'off,on,off,on,off,on,off,on'

cultivar = ds.createVariable('cultivar', 'f4', ('scen', 'lat', 'lon',))
cultivar.units = 'Mapping'
cultivar.long_name = 'GS,SS,FSPS,FS,GS,SS,FSPS,FS'
cultivar._DeflateLevel = 5

pdate = ds.createVariable('pdate', 'f4', ('scen', 'lat', 'lon',))
pdate.units = 'Julian day'
pdate.long_name = 'Planting date'
pdate._DeflateLevel = 5

attr = ds.setncattr('person_notes','Jonathan Ojeda')
attr = ds.setncattr('history','pSIMS setup for sorghum modelling in USA')

In [5]:
#Assign Latitude and Longitude Values
lats[:] = np.arange(-89.75, 90.25, 0.5)
lons[:] = np.arange(-179.75, 180.25, 0.5)

In [6]:
#Create an array for a variable (e.g. planting date in Julian day) with variable values accordingly with the lat/lon combination
#This array needs to be previously created based on user purposes

pdateArray = []
with open(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/pdateCampaignPsims.csv') as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader: # each row is a list
        pdateArray.append(row)

In [7]:
#Define name of scenarios and name and shape of variables (BE SURE YOU UPDATE THE NUMBER OF SCENARIOS TWICE!!!!)
for i in range(8):
    automatic_irrigation[i] = np.random.uniform(i+1,i+1, size=(8,))
    cultivar[i, :, :] = np.random.uniform(i+1,i+1, size=(360, 720))
    pdate[i, :, :] = pdateArray
    scens[i] = i+1
    
#Close the netCDF file
ds.close()

In [11]:
#Explore Netcdf
data = xr.open_dataset(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/data/campaign/created_campaign/test19.nc4', decode_times=False)

In [12]:
data

<xarray.Dataset>
Dimensions:               (lat: 360, lon: 720, scen: 8)
Coordinates:
  * lat                   (lat) float64 -89.75 -89.25 -88.75 ... 89.25 89.75
  * lon                   (lon) float64 -179.8 -179.2 -178.8 ... 179.2 179.8
  * scen                  (scen) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
Data variables:
    automatic_irrigation  (scen) int32 ...
    cultivar              (scen, lat, lon) float32 ...
    pdate                 (scen, lat, lon) float32 ...
Attributes:
    person_notes:  Jonathan Ojeda
    history:       pSIMS setup for sorghum modelling in USA